## 1.4. DQN
接下来我们来研究Q-Learning。我们首先来介绍两种值函数：状态值函数$v_{\pi}(s)$和状态行动值函数$q_{\pi}(s,a)$。在业界也将这两个函数称为Critic，即评判者，可以用于评价策略$\pi$的好坏，并据此对策略进行改进，也就是著名的Q-Learning，以及其对应的深度学习版本DQN。
### 1.4.1. 状态值函数
状态值函数$v_{\pi}(s)$表求在状态$s$可以获得的累积奖励的期望值：
$$
v_{\pi}=E(G_{t}|S_{t}=s)
$$
#### 1.4.1.1. Monte Carlo法
我们首先让Agent与环境互动$N$次，如玩$N$局游戏，对于每一局游戏：
$$
r_{1}^{1}, s_{1}^{1}, a_{1}^{1}, r_{2}^{1}, s_{2}^{1}, a_{2}^{1}, ...,r_{T_{1}}^{1},s_{T_{1}}^{1},a_{T_{1}}^{1}
$$
根据累称奖励定义：
$$
G_{t}=\sum_{k=0}^{\infty}\gamma ^{k} R_{t+k+1}
$$
我们可以求出每个时刻累积奖励：
$$
G_{1}^{1},G_{2}^{1},...,G_{T_{1}}^{1}
$$
与每一时刻状态$s_{t}$进行组合，可以形成如下数据集：
$$
(s_{1}^{1},G_{1}^{1}),(s_{2}^{1},G_{2}^{1}),...,(s_{T_{1}}^{1},G_{T_{1}}^{1})
$$
因此所有$N$个Trajectory形成一个数据集：
$$
(s_{1}^{1},G_{1}^{1}),(s_{2}^{1},G_{2}^{1}),...,(s_{T_{1}}^{1},G_{T_{1}}^{1}) \\
(s_{1}^{2},G_{1}^{2}),(s_{2}^{2},G_{2}^{2}),...,(s_{T_{1}}^{2},G_{T_{1}}^{2}) \\
...... \\
(s_{1}^{N},G_{1}^{N}),(s_{2}^{N},G_{2}^{N}),...,(s_{T_{1}}^{N},G_{T_{1}}^{N})
$$
对于每个样本，$s_{i}^{j}$代表在第$j$局中，第$i$个时间点的状态，为一个向量，如图像画面，$G_{i}^{j}$为其所对应的累积奖励，为一个标量数值。我们可以将其视为一个回归（Regression）任务，输入为向量$s_{i}^{j}$，输出值为$G_{i}^{j}$。

#### 1.4.1.2. Temporal Difference法
我们让Agent与环境互动，可以得到如下的Trajectory片断：
$$
......, s_{t}, a_{t}, r_{t+1}, s_{t+1}, ......
$$
根据状态值函数的定义可得：
$$
v_{\pi}(s_{t})=r_{t+1} + v_{\pi}(s_{t+1})
$$
将上式进行整理可得：
$$
v_{\pi}(s_{t}) - v_{\pi}(s_{t+1}) = r_{t+1}
$$
采用如下网络结构：
![TD法](./images/chp_01_04_01.png)

#### 1.4.1.3. MV vs TD
通过Monte Carlo和Temporal Difference方法，都可以通过回归算法（Regression）求出状态值函数。对于MC方法，如果数据量足够大，可以得到均值，但是会有比较大的方差。而TD方法则正好相反，其不具有逼近均值的能力，但是估计结果的方差较小。在实际使用中，TD方法会更常用一些。

### 1.4.2. 状态行动值函数
对于$q_{\pi}(s, a)$，我们在状态$s$时采用行动$a$，然后在其后的过程中，仍然使用策略$\pi$来选择行动，对于行动为离散型时：
![q](./images/chp_01_04_02.png)
如果行动是连续型的情况，则可以将状态$s$与所采取的行动一起，输入到网络，如下所示：
![q continue](./images/chp_01_04_03.png)

### 1.4.3. Q-Learning


下面我们来看Q-Learning的推导过程。我们假设有策略$\pi ^{*}$，满足：
$$
\pi ^{*}(s) = \arg \max_{a \in \mathcal{A}(s)} q_{\pi}(s, a) \\
\forall s \in \mathcal{S}: v_{\pi ^{*}}(s) \geqslant v_{\pi}(s)
$$
上式表明，在策略$\pi ^{*}$下，在状态$s$时，所采取的行动是使$q_{\pi}(s, a)$的值达到最大时的行动$a$。在策略$\pi ^{*}$下，对于所有的状态$s$，所取得的状态值函数的值均大于或等于在其他策略下状态值函数的值。
状态值函数可以视为是在状态$s$下根据策略$\pi$采取行动所获得的累积奖励，其一定小于或等于在状态$s$，采用最佳策略$\pi {*}$时采取的行动所获得的累积奖励，如下所示：
$$
v_{\pi}(s)=q_{\pi}(s, \pi(s)) \leqslant \max_{a \in \mathcal{A}(s)} q_{\pi}(s, a)=q_{*}(s, \pi ^{*}(s))
$$
将上式写为$q_{\pi}(s,a)$的定义式，如下所示：
$$
v_{\pi}(s) \leqslant q_{*}\Big( s, \pi ^{*}(s) \Big) = E \Big( r_{t+1} + v_{\pi}(s_{t+1})|S_{t}=s, A_{t}=\pi ^{*}(s) \Big)
$$
该值肯定小于在$t+1$时刻也采用最佳策略$\pi ^{*}$时采取的行动，如下所示：
$$
v_{\pi}(s_{t+1}) \leqslant  E \Big( r_{t+1} + v_{\pi}(s_{t+1})|S_{t}=s_{t}, A_{t}=\pi ^{*}(s_{t+1}) \Big) \\
\leqslant E \bigg( r_{t+1} + q_{\pi}(s_{t+1}, \pi ^{*}(s_{t+1}) | S_{t+1}=s_{t+1}, A_{t+1}=\pi ^{*}(s_{t+1}) \bigg) \\
= E \bigg( r_{t+1} + r_{t+2} + v_{\pi}(s_{t+2}) | S_{t+1}=s_{t+1}, A_{t+1}=\pi ^{*}(s_{t+1}) \bigg)  \\
...... \\
= E(r_{t+1} + r_{t+2} + ... + r_{T}) \\
= v_{\pi ^{*}}(s_{t}) = v_{*}(s_{t})
$$

#### 1.4.3.1. TD法
在实际的Q-Learning中，我们采用类似TD的方法。
我们首先让Agent与环境进行互动，得到如下Trajectory片断：
$$
..., s_{t}, a_{t}, r_{t+1}, s_{t+1}, ...
$$
根据定义有如下公式：
$$
q_{\pi}(s_{t}, a_{t})= r_{t+1} + q_{\pi}(s_{t+1}, \pi (s_{t+1}))
$$
式中$\pi (s_{t+1})$表示在状态$s_{t+1}$下由策略$\pi$来选择合适的行动。
我们采用如下的网络结构：
![target network](./images/chp_01_04_04.png)
在初始时，图中的Target NN与NN具有相同的参数$\boldsymbol{\theta}_{\pi}^{0}$，然后按照回归算法（Regression）来更新图中上面NN的参数，但是保持下面Target NN的参数不变，当运行$N$（超参数）次之后，用上面网络的参数$\boldsymbol{\theta}_{\pi}^{N}$去更新Target NN的参数。

#### 1.4.3.2. 探索
我们注意到，在Q-Learning中，策略$\pi$为：$a=\arg \max_{a \in \mathcal{A}(s)} q_{\pi}(s,a)$，其实际上没有探索的过程。在实际的算法中，我们通常会引入探索的成份。
* epsilon Grady算法
$$
a = \begin{cases} \arg \max_{a \in \mathcal{A}(s)} q_{\pi}(s, a), \quad p=1-\epsilon \\
random, \quad p=\epsilon
\end{cases}
$$
即其以$1-\epsilon$的概率取策略给定的行动，以$\epsilon$的概率随机取值。$\epsilon$的值随着训练的进行逐渐减小。
* Boltzman Exploration
我们定义取策略指定行动的概率为：
$$
p(a|s) = \frac{ \exp \big( q_{\pi}(s,a) \big) }{\sum_{a' \in \mathcal{A}(s)} \exp \big( q_{\pi}(s,a') \big) }
$$
在其他情况下随机取值。